In [1]:
# Connecting to Google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Importing required libraries

In [2]:
import os
from openai import OpenAI
from google.colab import userdata

In [3]:
# Setting Groq client with API key(api key stored in colab secrets)
api_key = userdata.get('OPENAI_API_KEY')
client = OpenAI(api_key=api_key, base_url="https://api.groq.com/openai/v1")

### Task-1: Conversation management + summarization

#### 1. Maintain running conversation history

In [4]:
# List to store conversations
conversations = []

In [5]:
# Function to add user-assisstant conversation
def add_message(role, content):
  conversations.append({'role':role,'content':content})

# Function to get history of conversation
def get_history_text():
  chat_history = "\n".join(f"{m['role'].capitalize()}: {m['content']}" for m in conversations)
  return chat_history

#### 2. Summarization function

In [6]:
def summarize_with_groq(text):
  response = client.chat.completions.create(
      model="meta-llama/llama-4-maverick-17b-128e-instruct",
      messages = [
          {"role":"system", "content":"You are a helpful summarizer."},
          {"role":"user", "content":f"Summarize the follwing conversation in 50 words or less:\n{text}"}
      ],
      max_tokens=200
  )

  return response.choices[0].message.content

#### 3. Truncation Options

In [7]:
# Truncate by runs
def truncate_by_runs(n):
  return conversations[-n:]

# Truncate by characters
def truncate_by_chars(max_chars):
  text = get_history_text()
  return text[-max_chars:]

#### 4. Periodic summarization

In [8]:
run_num = 0 # counter for runs
k = 3 # sumarize after every 3 runs

In [9]:
# Function to summarize every k runs and replace history with summary
def periodic_summarization():
  global run_num, k, conversations
  run_num +=1
  print(f"Run:{run_num}")

  if run_num % k == 0:
    chat_history = get_history_text()
    summary = summarize_with_groq(chat_history)
    print("**Summarization triggered**")
    print("Summary:", summary)

    #Replacing full conversation with summary
    conversations = [{"role":"system","content":f"Summary so far: {summary}"}]

  else:
    print("**No summarization**")

#### 5. Demonstration

In [10]:
# Feed multiple conversation samples
add_message("user", "Hi, I'm Vamshi. Can you help me?")
add_message("content", "Sure - what do you need help with?")
periodic_summarization()

add_message("user", "I want to book a flight for next week.")
add_message("content", "Great, from which city are you flying?")
periodic_summarization()

add_message("user", "From Hyderabad.")
add_message("content", "Got it. And what is your destination?")
periodic_summarization()

add_message("user", "I’m going to Bangalore.")
add_message("content", "Okay. Do you have specific travel dates in mind?")
periodic_summarization()

add_message("user", "Yes, next Monday morning.")
add_message("content", "Perfect. Do you prefer economy or business class?")
periodic_summarization()

add_message("user", "Economy class is fine.")
add_message("content", "Alright, I’ll check flights for Hyderabad to Bangalore next Monday in economy.")
periodic_summarization()

Run:1
**No summarization**
Run:2
**No summarization**
Run:3
**Summarization triggered**
Summary: Vamshi wants to book a flight for next week from Hyderabad. The conversation has just begun, with the destination still to be determined.
Run:4
**No summarization**
Run:5
**No summarization**
Run:6
**Summarization triggered**
Summary: Vamshi wants to book a flight from Hyderabad to Bangalore next Monday morning in economy class. The conversation is ongoing to check available flights.


### Task-2: JSON Schema classification and extraction

#### 1. Creating a JSON Schema

In [11]:
user_schema = {
    "name": {"type": "string", "description": "Full name of the user"},
    "email": {"type": "string", "description": "Email address"},
    "phone": {"type": "string", "description": "Phone number"},
    "location": {"type": "string", "description": "City or place"},
    "age": {"type": "integer", "description": "Age in years"}
}

#### 2. OpenAI function calling with Groq API

In [21]:
def extract_info(text):
  response = client.chat.completions.create(
      model="meta-llama/llama-4-maverick-17b-128e-instruct",
      messages=[
            {"role": "system", "content": "You are an information extraction assistant."},
            {"role": "user", "content": f"""
            Extract the following details ONLY as valid JSON with this schema:
            {{
              "name": "string",
              "email": "string",
              "phone": "string",
              "location": "string",
              "age": integer
            }}

            Strict rules:
            - Do NOT include extra keys or formatting.
            - Do NOT return text outside JSON.
            - Ensure age is an integer, not a string.

            Text: {text}
            """}
      ],
      max_tokens=200
  )

  return response.choices[0].message.content.strip()

#### 3. Demonstration

##### a. Parsing sample chats

In [19]:
sample_chats = [
    "Hi, I’m Vamshi Ramagoni from Hyderabad. You can email me at vamshiramagoni3@gmail.com or call me at +91 9999999999. I am 21 years old.",
    "Hello, my name is Alice. I live in New York. My email is alice@example.com and my phone number is 555-1234. I’m 30 years old.",
    "Hey, I’m Bob from London. You can reach me at bob123@gmail.com or on my phone +44 7700 900123. I’m 25 years old."
]


In [20]:
# Extracting detials from sample chats
for chat in sample_chats:
  json_info = extract_info(chat)
  print("Extracted INFO:", json_info)


Extracted INFO: {"name": "Vamshi Ramagoni", "email": "vamshiramagoni3@gmail.com", "phone": "+919999999999", "location": "Hyderabad", "age": 21}
Extracted INFO: {"name": "Alice", "email": "alice@example.com", "phone": "555-1234", "location": "New York", "age": 30}
Extracted INFO: {"name": "Bob", "email": "bob123@gmail.com", "phone": "+447700900123", "location": "London", "age": 25}


##### Validation of O/P's

In [22]:
import json

# Validation Function
def validate_json(json_text):
  try:
    data = json.loads(json_text)
    for field in user_schema.keys():
      if field not in data:
        return False, f"Missing field: {field}"

    return True, data

  except Exception as e:
    return False, f"Invalid JSON: {str(e)}"



In [23]:
# Apply validation
for chat in sample_chats:
  info_json = extract_info(chat)
  is_valid,result = validate_json(info_json)
  if is_valid:
        print("Valid JSON:", result)
  else:
        print("Validation error:", result)

Valid JSON: {'name': 'Vamshi Ramagoni', 'email': 'vamshiramagoni3@gmail.com', 'phone': '+919999999999', 'location': 'Hyderabad', 'age': 21}
Valid JSON: {'name': 'Alice', 'email': 'alice@example.com', 'phone': '555-1234', 'location': 'New York', 'age': 30}
Valid JSON: {'name': 'Bob', 'email': 'bob123@gmail.com', 'phone': '+447700900123', 'location': 'London', 'age': 25}
